No primeiro notebook vimos alguns conceitos de manipulação de dados, como remover linhas, substituir nulos, plotar gráficos, entre outros. Todos esses conceitos são essenciais para termos um conjunto de dados limpo e pronto para colocar dentro de um modelo de machine learning com o objetivo de predizer algo. 

Porém, só com os conceitos aprendidos de limpeza, não conseguimos atingir o objetivo da predição de uma variável. Dessa forma, esse notebook aborda o início, que é a análise dos dados, o meio, que é a modelagem dos dados(limpeza, transformação e criação de features) e o fim que é a predição dos dados.

A ideia desse notebook é vir como um tutorial que possa dar clareza sobre os processos. 

Depois desse notebook terá mais um tutorial da serie Titanic, porém, com análises e raciocínios mais complexos. Lembrando que o objetivo é sempre ter um modelo de predição com altíssima taxa de acerto.

## **Importando Libraries Necessárias para o desenvolvimento da análise**

In [ ]:
import warnings
warnings.filterwarnings("ignore") #Para não poluir o notebook com avisos 

import pandas as pd #Manipulação de Dados
import numpy as np #Álgebra Linear
import matplotlib.pyplot as plt #Visualização de Dados
import seaborn as sns #Visualização de Dados
import pandas_profiling #relatório pronto para entender os dados

# machine learning

from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


## **Lendo Arquivos**

In [ ]:
test = pd.read_csv("../input/titanic/test (1).csv")
train = pd.read_csv("../input/titanic/train (1).csv")
combine = [train, test]


# **ANÁLISE DE DADOS**

Vamos aqui entender os dados que temos no nosso conjunto e assim escolher quais as features(atributos ou colunas) iremos manter. Essa escolha de features é essencial para o modelo, uma vez que se eu escolho features que não tem nenhuma ligação com a minha label (variável que irei prever), então não importa o modelo que eu use, sempre terei uma taxa pífia de acerto.

# **Vamos ver os datasets em mais detalhe**


In [ ]:
print(train.shape)
test.shape

In [ ]:
train.dtypes

In [ ]:
test.dtypes

# **Vamos focar no dataset de Treino e depois replicamos as mudanças no Teste**

In [ ]:
profile_report = pandas_profiling.ProfileReport(train)
profile_report

In [ ]:
train.describe(include=[np.number]).T

In [ ]:
train.describe(include='O').T


**Vamos ver as colunas que possuem nulo**

In [ ]:
pd.options.display.max_rows = 100 ##Caso a visualização abaixo fique truncada, é só rodar essa linha de código


In [ ]:
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = round(df.isnull().sum().sort_values(ascending=False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    return temp.loc[temp['Total'] > 0]

In [ ]:
missingValuesInfo(train)

In [ ]:
#Cabin
train[['Cabin', 'Survived']].groupby(['Cabin'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
plt.figure(figsize=(3,3))

train.Cabin.value_counts().plot(kind='bar')
plt.ylabel('Counts')
plt.xlabel('Cabin')
#plt.title('Gender Distribution')
plt.show()

Olhando Cabin, não vejo nada que posso associar com a label Survived. Além de ter muitos nulos e os valores restantes são majoritariamente únicos. Remover

In [ ]:
#PCLASS
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
plt.figure(figsize=(3,3))

train.Pclass.value_counts().plot(kind='bar')
plt.ylabel('Counts')
plt.xlabel('Pclass')
#plt.title('Gender Distribution')
plt.show()

Tenho aqui uma visão de que quem ficou na primeira classe teve uma sobrevivência mais alta, o que já me diz algo com a label Survived. Manter

In [ ]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Sexo tem um padrão bem forte de sobrevivência feminina, o que me diz bastante coisa relacionada à label survived. Vamos manter.

In [ ]:
train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Sibs tem muitos zeros, mas eles representam viajantes solitários, o que é interessante, uma vez que 0, 1 e 2 tem boa relação com a label Survived. Vou manter

In [ ]:
train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Vou manter porque vejo relação de 60% Parch 3. Não entendo bem essa feature, mas vou manter.

In [ ]:
#Embarked
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Por mais que a maioria dos viajantes tenham embarcado pelo S, a maior taxa de sobrevivência está no C, então achei interessante e vou manter.

**Elimino as linhas em que forem nulas na feature principal de predição do treino,
porque isso pode impactar negativamente no modelo de ia! (nem preciso ver se tem nulo ou não)**

In [ ]:
Target = 'Survived'
train.dropna(axis=0, subset =[Target], inplace=True)

**Visões**

Fare - manter

SibSp - manter ( agrupar)

Parch - manter

Ticket has a high cardinality: 681 distinct values Warning - remover

Cabin - 687 de 891 desconhecido  - remover

Embarked tem alta cardinalidade com 644 de 891 sendo S, porém, a maioria dos que sobrevivem são do C. Achei interessante, então vou manter.

Name - 891 de 891 únicos, não me dando nenhuma característica, entao eu removo

PassengerId      Manter (Importante para o modelo ter como índice) 

Sex             Manter

Age            Manter


# **MODELAGEM DE DADOS**

# **Remover as colunas que identificamos que não irão participar do modelo**

In [ ]:
def dropValues(df):
  return df.drop(['Name', 'Cabin', 'Name','Ticket'],axis = 1)

In [ ]:
train = dropValues(train)

In [ ]:
train.shape

# **Lidando com Missing Data - Nulos!**


In [ ]:
def HandleMissingValues(df):
    # for Object columns fill using 'UNKOWN'
    # for Numeric columns fill using median
    num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
    cat_cols = [cname for cname in df.columns if df[cname].dtype == "object"]
    values = {}
    for a in cat_cols:
        values[a] = 'S' #Somente 2 valores nulos e vou colocar o que é mais frequente. 
    for a in num_cols:
        values[a] = df[a].median()

    df.fillna(value=values,inplace=True)   

In [ ]:
missingValuesInfo(train)

In [ ]:
HandleMissingValues(train)

**Validando que lidamos com os dados nulos**

In [ ]:
missingValuesInfo(train)

In [ ]:
train.head(3)

# **Olhando novamente as colunas**

In [ ]:
train.dtypes.T

In [ ]:
train.describe(include='O')

# **Transformar String em Int**




In [ ]:
def getObjectColumnsList(df):
    return [cname for cname in df.columns if df[cname].dtype == "object"]

def PerformOneHotEncoding(df, columnsToEncode):
    return pd.get_dummies(df, columns = columnsToEncode)

In [ ]:
cat_cols = getObjectColumnsList(train)

In [ ]:
print(cat_cols)

In [ ]:
train = PerformOneHotEncoding(train, cat_cols)

# **Vamos dar aquela olhada final nas colunas restantes!**

In [ ]:
train.dtypes.T

In [ ]:
train.head()

# **Transformar Float em INT**

In [ ]:
train['Age'] = round(train['Age']).values.astype(np.int64)

In [ ]:
train.dtypes

In [ ]:
train.head()

# **Vamos fazer no dataset TEST agora**

Fizemos todas as alterações no dataset de train, agora vamos fazer todas as alterações no dataset de test para deixar tudo igual.

In [ ]:
HandleMissingValues(test)

test = dropValues(test)

cat_colsTest = getObjectColumnsList(test)

test = PerformOneHotEncoding(test, cat_colsTest)

test['Age'] = round(test['Age']).values.astype(np.int64)
#test['Fare'] = round(test['Fare']).values.astype(np.int64)
#test['Sex_female'] = round(test['Sex_female']).values.astype(np.int64)
#test['Sex_male'] = round(test['Sex_male']).values.astype(np.int64)

# **Olhada Final antes de ir para Machine Learning**

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
print(train.shape)
test.shape

# **Vamos modelar os datasets para aplicar Machine Learning**

In [ ]:
train_df = train.drop(['PassengerId'], axis=1)
test_df = test
combine = [train_df, test_df]
train_df.shape, test_df.shape

In [ ]:
X_train = train_df.drop("Survived", axis=1) #features
Y_train = train_df["Survived"] # label

X_test  = test_df.drop("PassengerId", axis=1).copy() #test

X_train.shape, Y_train.shape, X_test.shape

# **PREDIÇÃO DOS DADOS**

Nessa fase, temos os conjuntos de dados prontos, separados e preparados para rodar nos modelos de Machine Learning. Agora, vamos rodar em vários modelos e ver qual vai gerar uma resposta melhor.

# **Aplicando modelo de Machine Learning**

São vários modelos que você pode escolher e ai fica como lição de casa qual seria o melhor modelo e como seria adaptar outros modelos aqui ;p
Vou deixar o Ridge como exemplo, porque rodou bem mais rápido que os outros modelos. Importante saber que outros modelos chegam a fornecer acurácia de 99% sem stress.

In [ ]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier().fit(X_train, Y_train)
clf.score(X_train, Y_train)


Modelo rodou, gostei do resultado, vou modelar o output: 'PassengerId' e 'Survived' e depois vou exportar para csv e assim submeter no kaggle;

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": clf
    })


In [ ]:
submission.to_csv('submission.csv', index=False)